#librairies

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from urllib.parse import urljoin
from lxml import html
from tqdm import tqdm
import re
from datetime import datetime
import hashlib
import sqlite3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
url = "https://www.cadremploi.fr/emploi/liste_offres?motscles=data"
html = requests.get(url)

In [4]:
soup = BeautifulSoup(html.text, 'html.parser')

# MULTI-PAGES

In [5]:
# urls des annonces des 30 premieres pages
def get_all_links(base_url, num_pages):
    data = []
    domain = "https://www.cadremploi.fr"

    for page_num in tqdm(range(1, num_pages + 1)):
        url = f"{base_url}{page_num}"
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            for offer in soup.find_all('a', class_='row js-lien-detail-offre offre'):
                href = urljoin(domain, offer.get('href'))
                date_block = offer.find_next('div', class_='col-xs-12 derniere-ligne')
                date_text = date_block.get_text().strip()

                entry = {
                    'url': href,
                    'publication': date_text
                }
                data.append(entry)
        else:
            print(f"Erreur lors de la récupération de la page {page_num}")

    df = pd.DataFrame(data)
    return df

In [6]:
df_jobs = get_all_links("https://www.cadremploi.fr/emploi/liste_offres?motscles=data&page=", 30)

100%|██████████| 30/30 [00:45<00:00,  1.51s/it]


In [7]:
# determiner les annonces publié il y a moins de 24h
def annonces_moins_de_24h(df):
    cadremploi_24h = df[df['publication'].str.contains("Publiée il y a moins de 24h")]
    cadremploi_24h = cadremploi_24h.drop(columns=['publication'])
    return cadremploi_24h

In [8]:
cadremploi_24h = annonces_moins_de_24h(df_jobs)

In [31]:
# recuperer les infos des offres de moins de 24h
def scrape_pages(df_jobs):
    data = []
    site_annonce = "cadre emploi"
    compteur_merge = 0
    df_merged = pd.DataFrame()
    for index, row in tqdm(df_jobs.iterrows(), total=len(df_jobs), desc="Scraping jobs"):
        url = row['url']
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                entreprise = soup.find('div', class_='detail-nom-entreprise').text.strip()
                publication_time = soup.find('span', class_='date-publication').find('time')['datetime']
                publication = datetime.strptime(publication_time, '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%d-%m-%Y')
                poste = soup.find('span', class_='position titre-offre').text.strip()
                contrat = soup.find('div', class_='detail-type-contrat-entreprise').text.strip()
                profil = soup.find('p', class_='desc__p desc_profil cache').text.strip()
                description = soup.find('p', class_='desc__p desc_missions cache').text.strip()
                ville = soup.find('div', class_='detail-localisation-entreprise').text.strip()

                entry = {
                    'site_annonce': site_annonce,
                    'entreprise': entreprise,
                    'publication': publication,
                    'poste': poste,
                    'contrat': contrat,
                    'profil': profil,
                    'description': description,
                    'ville': ville,
                    'url': url
                }

                data.append(entry)
        except:
            pass
        df = pd.DataFrame(data)
    return df

In [32]:
df_all_jobs = scrape_pages(cadremploi_24h)

Scraping jobs: 100%|██████████| 360/360 [08:12<00:00,  1.37s/it]


In [33]:
df_all_jobs.to_csv('/content/drive/MyDrive/wild code school/Projets/Projet_3/scrapping/cadre_emploi/df_cadremploi_not_clean.csv', index = False)

# nettoyage

In [34]:
df = pd.read_csv('/content/drive/MyDrive/wild code school/Projets/Projet_3/scrapping/cadre_emploi/df_cadremploi_not_clean.csv')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   site_annonce  322 non-null    object
 1   entreprise    322 non-null    object
 2   publication   322 non-null    object
 3   poste         322 non-null    object
 4   contrat       322 non-null    object
 5   profil        322 non-null    object
 6   description   322 non-null    object
 7   ville         272 non-null    object
 8   url           322 non-null    object
dtypes: object(9)
memory usage: 22.8+ KB


In [35]:
# nettoyer et filtrer le csv
def clean_data(df):
    df = df.apply(lambda x: x.str.lower())
    df = df[df.poste.str.contains("data")]
    df = df.drop_duplicates(subset=['url'])
    df['poste'] = df.poste.str.replace(r'\s+nouveau$', '', regex=True)
    df['contrat'] = df['contrat'].str.replace(r'^apprentissage/alternance', 'alternance', regex=True)
    return df

In [36]:
df2 = clean_data(df_all_jobs)

In [37]:
print(df2.to_markdown())

|     | site_annonce   | entreprise                                       | publication   | poste                                                              | contrat            | profil                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [38]:
# generer une colonne id
def generate_id(row, columns):
    # Concaténer les valeurs des colonnes sélectionnées en une seule chaîne
    value = ''.join(str(row[col]) for col in columns)
    # Générer un hachage MD5 de cette chaîne
    return hashlib.md5(value.encode()).hexdigest()

In [39]:
def generate_and_replace_id(df, columns_to_hash, old_id_column):
    df['id'] = df_copy.apply(lambda row: generate_id(row, columns_to_hash), axis=1)
    df.drop(columns=[old_id_column], inplace=True)
    return df_copy

In [40]:
df2['id'] = df2.apply(lambda row: generate_id(row, ["entreprise", "poste" , "contrat" , "ville"]), axis=1)

In [41]:
ordre_final = [ 'id','site_annonce','entreprise', 'publication', 'poste', 'contrat', 'profil', 'description', 'ville', 'url']
df2 = df2[ordre_final]

In [48]:
df2['publication'] = pd.to_datetime(df2['publication'],dayfirst=True)

In [49]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 0 to 150
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            33 non-null     object        
 1   site_annonce  33 non-null     object        
 2   entreprise    33 non-null     object        
 3   publication   33 non-null     datetime64[ns]
 4   poste         33 non-null     object        
 5   contrat       33 non-null     object        
 6   profil        33 non-null     object        
 7   description   33 non-null     object        
 8   ville         33 non-null     object        
 9   url           33 non-null     object        
dtypes: datetime64[ns](1), object(9)
memory usage: 2.8+ KB


In [50]:
df2.to_csv("/content/drive/MyDrive/wild code school/Projets/Projet_3/scrapping/cadre_emploi/df_cadremploi_clean.csv", index = False)

In [51]:
print(df2.to_markdown())

|     | id                               | site_annonce   | entreprise                                       | publication         | poste                                                              | contrat            | profil                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [52]:
def db_file_storage(db_name, table_name, df):

    # Connexion ou création du fichier .db
    conn = sqlite3.connect(f"{db_name}.db")

    # Enregistrer le DataFrame dans le fichier
    df.to_sql(name=table_name, con=conn, if_exists='append', index=False)

    # Fermer la connexion
    conn.close()

In [53]:
db_file_storage("cadremploi","df_clean",df)